Graphs to make:
- [ ] 1. Conversion in function of SEI
- [ ] 2. Product yield in function of conversion with selectivity as slope
- [ ] 5. Atom balances in function of SEI
- [ ] 7. Product yield in function of SEI

Step 0: Importing libraries and creating static variables

In [ ]:
# Importing the libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import scipy.stats as stats

import warnings
from numpy import RankWarning

# Suppress RankWarnings
warnings.filterwarnings('ignore', category=RankWarning)

In [ ]:
# Define the alpha value for the post-plasma materials
alpha = 0.65

# Define the color for each material
packing_colors = {
    'γ-Al₂O₃': (0/255, 0/255, 0/255),
    '2% CuO@γ-Al₂O₃': (86/255, 180/255, 233/255),
    '10% CuO@γ-Al₂O₃': (0/255, 114/255, 178/255),
    '2% Fe₂O₃@γ-Al₂O₃': (230/255, 159/255, 0/255),
    '10% Fe₂O₃@γ-Al₂O₃': (213/255, 94/255, 0/255)
}

# determine order of materials for plotting
packing_order = [
    'γ-Al₂O₃',
    '2% CuO@γ-Al₂O₃',
    '2% Fe₂O₃@γ-Al₂O₃',
    '10% CuO@γ-Al₂O₃',
    '10% Fe₂O₃@γ-Al₂O₃'
]

t_quantile = stats.t.ppf(1 - 0.05/2, 7)

In [ ]:
# Use Times New Roman font
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Times New Roman'
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.rm'] = 'Times New Roman'
plt.rcParams['mathtext.it'] = 'Times New Roman:italic'
plt.rcParams['mathtext.bf'] = 'Times New Roman:bold'

# Set font size
plt.rcParams['font.size'] = 7

Step 1: Importing & preparing data

In [ ]:
# Importing the dataset
# set the folder and file name
folder_path = r'N:\FWET\FDCH\AdsCatal\General\personal_work_folders\plasmacatdesign\co2-splitting\uhasselt'
file_path = os.path.join(folder_path, 'uhasselt_co2_splitting_reaction_data_combined.csv')

# read the Excel file
df = pd.read_csv(file_path)

In [ ]:
# Preparing the data for all plots
df = df.query(
    'material in ["HT Glass Beads", "GM10.3", "GM10.4", "GM11.2", "SiO2+TMAH+2-PrOH-220/09H", "SiO2+TMAH-220/03H"]'
).assign(
    compound=lambda x: x['compound'].map(
        {
            'CO2': 'CO₂',
            'CO': 'CO',
            'O2': 'O₂',
            'N2': 'N₂'
        }
    ),
    material=lambda x: x['material'].map(
        {
            
            'HT Glass Beads': "SiO₂ - HT Etched",
            'GM10.3': 'GM10.3',
            'GM10.4': 'GM10.4',
            'GM11.2': 'GM11.2',
            'SiO2+TMAH+2-PrOH-220/09H': 'SiO₂ - TMAH/2-PrOH 9 h Etched',
            'SiO2+TMAH-220/03H': 'SiO₂ - TMAH 3 h Etched',
        }
    )
)

Step 2: Conv vs SEI

In [ ]:
# Prepare the data for the conversion plotting
df_conv = df[
    [
        'packing',
        'compound',
        'res_time_sec',
        'conv',
        'conv_sd'
    ]
].query(
    "compound in ['CO₂']"
).drop_duplicates(
).assign(
    conv=lambda x: x['conv'] * 100,
    conv_sd=lambda x: x['conv_sd'] * 100,
    sei_ideal_kjmol=lambda x: round(
        number=30/((1-0.4774)*(17.31/x['res_time_sec']))*24.055,
        ndigits=-1
    ).astype(int)
).groupby(
    ['packing', 'res_time_sec', 'sei_ideal_kjmol']
).agg(
    conv_mean=('conv', 'mean'),
    conv_sd=('conv_sd', lambda x: ((x**2).sum()**0.5)/len(x))
).reset_index(
).copy()

In [ ]:
# Plot the conversion vs SEI
g = sns.lmplot(
    data=df_conv,
    x='sei_ideal_kjmol',
    y='conv_mean',
    hue='packing',
    palette=packing_colors,
    hue_order=packing_order,
    height=5/2.54,
    aspect=1,
    order=2,
    ci=None,
    legend=True,
    scatter_kws={'s': 8, 'alpha': alpha},
    line_kws={'lw': 0.9}
).set(
    xlabel='SEI (kJ mol⁻¹)',
    ylabel='CO$_2$ Conversion (%)'
)

g.legend.set_title('Packing', prop={'size': 8})

ax = g.axes.flatten()[0]

for packing in df_conv['packing'].unique():
    subset = df_conv[df_conv['packing'] == packing]
    ax.errorbar(
        x=subset['sei_ideal_kjmol'],
        y=subset['conv_mean'],
        yerr=subset['conv_sd'] * t_quantile,
        fmt='none',
        ecolor=packing_colors[packing],
        elinewidth=0.5,
        capsize=1,
        capthick=0.95,
        alpha=0.6
    )

plt.savefig(
    r"C:\Users\sbossier\Dropbox\Apps\Overleaf\ladca_desine_paper_etching\figs\sander_conv_vs_sei.pdf",
    bbox_inches='tight'
)
plt.show()